<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/json_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd

incorrect_player_name_to_correct_player_name_map = {
  "A.J. Dillon": "AJ Dillon",
  "Nathaniel Dell": "Tank Dell",
  "Josh Palmer": "Joshua Palmer",
  "Devon Achane": "De'Von Achane",
  "Gabriel Davis": "Gabe Davis",
  "Devonta Smith": "DeVonta Smith",
  "D.J. Moore": "DJ Moore",
  "Odell Beckham": "Odell Beckham Jr.",
  "Richie James": "Richie James Jr.",
  "Jeff Wilson": "Jeff Wilson Jr.",
  "Jeff Wilson Jr": "Jeff Wilson Jr.",
  "Irv Smith": "Irv Smith Jr.",
  "Irv Smith Jr": "Irv Smith Jr.",
  "Michael Pittman": "Michael Pittman Jr.",
  "Michael Pittman Jr": "Michael Pittman Jr.",
  "Travis Etienne": "Travis Etienne Jr.",
  "Travis Etienne Jr": "Travis Etienne Jr.",
  "Brian Robinson": "Brian Robinson Jr.",
  "Brian Robinson Jr": "Brian Robinson Jr.",
  "Chig Okonkwo": "Chigoziem Okonkwo",
  "DJ Chark Jr": "DJ Chark",
  "DJ Chark Jr.": "DJ Chark",
  "D.J. Chark Jr.": "DJ Chark",
  "Terrace Marshall": "Terrace Marshall Jr.",
  "Terrace Marshall Jr": "Terrace Marshall Jr.",
  "Allen Robinson II": "Allen Robinson",
  "Calvin Austin III": "Calvin Austin",
  "Melvin Gordon III": "Melvin Gordon",
  "Marvin Jones": "Marvin Jones Jr.",
  "Marvin Jones Jr": "Marvin Jones Jr.",
  "N'Keal Harry": "N'Keal Harry",
  "Laviska Shenault": "Laviska Shenault Jr.",
  "Laviska Shenault Jr": "Laviska Shenault Jr."
}

def clean_player_name(player_name):
  correct_player_name = incorrect_player_name_to_correct_player_name_map.get(player_name, "")

  if correct_player_name == "":
        return player_name

  return correct_player_name

class Position:
  def __init__(self, position, whenOutIncreaseUsagePositions):
      self.position = position
      #self.handcuff_positions = handcuffPositions
      self.when_out_increase_usage_positions = whenOutIncreaseUsagePositions

class Player:
    def __init__(self, position, name):
        self.position = position
        self.name = name

class TeamRoster:
    def __init__(self, team_abbrev, players, positions):
        self.team_abbrev = team_abbrev
        self.players = players
        self.positions = positions

def get_team_roster(team):
  endpoint_url = f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/teams/{team.lower()}.json"
  team_roster_json = ""

  try:
      # Send an HTTP GET request to the endpoint
      response = requests.get(endpoint_url)

      # Check if the request was successful (status code 200)
      if response.status_code == 200:
          # Parse the JSON response
          team_roster_json = json.dumps(response.json())

          # Now you can work with the JSON data as a Python dictionary
          # print(team_roster_json)

      else:
          # Handle the case where the request was not successful
          print(f"Request failed with status code {response.status_code}")

  except requests.exceptions.RequestException as e:
      # Handle any network-related exceptions
      print(f"An error occurred: {e}")
  except json.JSONDecodeError as e:
      # Handle JSON decoding errors
      print(f"Failed to parse JSON: {e}")

  #print(team_roster_json)

  # Deserialize the JSON data into a Python dictionary
  data_dict = json.loads(team_roster_json)

  positions = [Position(**position_data) for position_data in data_dict["positions"]]

  players = [Player(**player_data) for player_data in data_dict["players"]]

  team_roster = TeamRoster(data_dict["teamAbbrev"], players, positions)

  return team_roster

#print(team_roster.team_abbrev)
#print(len(team_roster.players))

def get_replacement_players(team_roster, out_player_position):
  replacement_players = []
  for position in team_roster.positions:
    if position.position == out_player_position:
      for when_out_increase_usage_position in position.when_out_increase_usage_positions:
        for player in team_roster.players:
          if player.position == when_out_increase_usage_position:
            if player.name not in replacement_players:
              replacement_players.append(f"{player.name} ({player.position})")
  return replacement_players

def format_players(players):
  delimiter = ", "

  return delimiter.join(players)

def get_week_salary(all_players_df, week, player_name):
  filtered_df = all_players_df[all_players_df["Name"] == player_name]

  if len(filtered_df) > 0:
    return filtered_df[f'Week{week}_Salary'].iloc[0]

  return 0

def format_player_info(player_name, position, salary):
  return f"{position} {player_name} ${salary}"

def get_games(week, start_date, end_date):
  games_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DKSalaries-week{week}.csv")

  games_df = games_df.drop(columns=['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame'])

  split_columns = ['Game', 'Date', 'Time', 'TimeZone']
  games_df[split_columns] = games_df['Game Info'].str.split(" ", expand=True)
  games_df = games_df.drop_duplicates(subset=['Game Info', 'Game', 'Date', 'Time', 'TimeZone'])
  games_df = games_df.sort_values(by=['Date', 'Time', 'Game'], ascending=[True, True, True])
  games_df[['Away Team', 'Home Team']] = games_df['Game'].str.split("@", expand=True)

  games_df['Datetime'] = pd.to_datetime(games_df['Date'] + ' ' + games_df['Time'])

  games_df = games_df[(games_df['Datetime'] >= start_date) & (games_df['Datetime'] <= end_date)]
  games_df = games_df.sort_values(by=['Date', 'Time', 'Game'], ascending=[True, True, True])

  split_columns.append('Datetime')
  games_df = games_df.drop(columns=split_columns)

  #print(games_df)

  return games_df

def add_player_row(team_players_df, week, player_name, position, salary, row_index):
  row = {'Position': [position], f'Week {week} Salary': [salary], 'Name': [player_name]}
  team_players_df = pd.concat([team_players_df, pd.DataFrame(row, index=[row_index])], axis=0, ignore_index=True)

def print_team(week, team_abbrev, all_players_df, injury_report_df, player_names):
  team_roster = get_team_roster(team_abbrev)
  row_index = 0
  team_players_df = pd.DataFrame(columns=['Position', f'Week {week} Salary', 'Name'])

  for player in team_roster.players:
    player_names.append(player.name)

  team_condition = injury_report_df["Team"] == team_roster.team_abbrev
  team_injury_report_df = injury_report_df[team_condition]

  player_names_condition = team_injury_report_df["Player"].isin(player_names)
  team_dfs_players_injury_report_df = team_injury_report_df[player_names_condition]
  # Player	Team	Pos	Injury	Status	Est. Return

  for player in team_roster.players:
    salary = get_week_salary(all_players_df, week, player.name)
    player_info = format_player_info(player.name, player.position, salary)

    print(player_info)

    if len(team_dfs_players_injury_report_df) > 0:
      hasInjury = False
      for index, injury_row in team_dfs_players_injury_report_df.iterrows():
        if player.name == injury_row["Player"]:
          print(f"    Injury: {injury_row['Status']} - {injury_row['Injury']}")

          if injury_row["Status"] != "Questionable":
            replacement_players = format_players(get_replacement_players(team_roster, player.position))
            if len(replacement_players) > 1:
              print(f"    Replacement(s): {replacement_players}")

  if len(team_injury_report_df) > 0:
    #position_sort_order = ["C", "OT", "OG", "DE", "DT", "LB", "CB", "S", "K"]
    positions = ["QB", "RB", "WR", "TE"]
    estimated_return_condition = team_injury_report_df["Est. Return"] < pd.to_datetime('2024-02-12')
    positions_condition = ~team_injury_report_df["Pos"].isin(positions)

    team_injury_report_df = team_injury_report_df[positions_condition & estimated_return_condition]
    sorted_df = team_injury_report_df.sort_values(by=["Pos"], ascending=[True])

    print("")
    print("Other Injuries")
    print(sorted_df.to_string(index=False))

def get_injury_report(week):
  injury_report_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/nfl-injury-report-week{week}.csv")

  for row in injury_report_df.index:
    injury_report_df["Est. Return"] = pd.to_datetime(injury_report_df.at[row, "Est. Return"])
    injury_report_df.at[row, "Player"] = clean_player_name(injury_report_df.at[row, "Player"])

  return injury_report_df

def get_defense_man_zone_coverage(week):
  coverage_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/defense-man-zone-coverage.csv")

  nbr_of_weeks_to_return = 5
  start_week = 1

  if (week > nbr_of_weeks_to_return):
    start_week = week - nbr_of_weeks_to_return

  weeks = [start_week for start_week in range(nbr_of_weeks_to_return)]

  week_condition = coverage_df["Week"].isin(weeks)
  coverage_df = coverage_df[week_condition].sort_values(by=['Week'], ascending=[False])

  return coverage_df

def print_defense_man_zone_coverage(coverage_df, team_abbrev):
  team_condition = coverage_df['Team'] == team_abbrev
  coverage_df = coverage_df[team_condition]

  for index, coverage_row in coverage_df.iterrows():
    print(f"Week {coverage_row['Week']} Man Coverage rank: {coverage_row['ManCoverageRank']} ({coverage_row['ManCoveragePercent']}%)")
    print(f"       Zone Coverage rank: {coverage_row['ZoneCoverageRank']} ({coverage_row['ZoneCoveragePercent']}%)")

#***** update each week ******
week = 15
contest_start_date = '12/16/2023 01:00PM'
contest_end_date = '12/16/2023 09:30PM'
#************************

all_players_df = pd.read_csv("https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-Players.csv?1=2")
injury_report_df = get_injury_report(week)
coverage_df = get_defense_man_zone_coverage(week)
games_df = get_games(week=week, start_date=contest_start_date, end_date=contest_end_date)

for index, game_row in games_df.iterrows():
  player_names = []

  print('')
  print(game_row['Game Info'])
  away_team = game_row['Away Team']
  home_team = game_row['Home Team']

  print('')
  print(away_team)
  print_defense_man_zone_coverage(coverage_df, away_team)
  print('')
  print_team(week, away_team, all_players_df, injury_report_df, player_names)

  print('')
  print(home_team)
  print_defense_man_zone_coverage(coverage_df, home_team)
  print('')
  print_team(week, home_team, all_players_df, injury_report_df, player_names)

  #team_injury_report_df.tail()


MIN@CIN 12/16/2023 01:00PM ET

MIN
Week 4 Man Coverage rank: 21 (23.0%)
       Zone Coverage rank: 14.0 (75.6%)
Week 3 Man Coverage rank: 21 (23.0%)
       Zone Coverage rank: 14.0 (75.6%)
Week 2 Man Coverage rank: 31 (6.2%)
       Zone Coverage rank: 2.0 (91.7%)
Week 1 Man Coverage rank: 31 (5.9%)
       Zone Coverage rank: 5.0 (89.7%)

QB1 Kirk Cousins $4000.0
    Injury: IR - Achilles
    Replacement(s): Nick Mullens (QB2)
QB2 Nick Mullens $5000.0
RB1 Alexander Mattison $5400.0
    Injury: Out - Ankle
    Replacement(s): Cam Akers (RB2), Ty Chandler (RB3)
RB2 Cam Akers $4000.0
    Injury: IR - Achilles
    Replacement(s): Alexander Mattison (RB1), Ty Chandler (RB3)
RB3 Ty Chandler $5300.0
WR1 Justin Jefferson $8100.0
    Injury: Questionable - Chest
WR2 Jordan Addison $5600.0
WR3 K.J. Osborn $4200.0
TE1 T.J. Hockenson $5800.0
TE2 Josh Oliver $2500.0

Other Injuries
          Player Team Pos      Injury       Status Est. Return
      Dean Lowry  MIN  DE    Pectoral           IR  202